In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"

Installed Packages Newtonsoft.Json, 13.0.1

In [ ]:
using Newtonsoft.Json;
using System.Text.RegularExpressions;

class Game {
    public string Name;
    public string Cover;
}

class Console {
    public string Name;
    public List<Game> Games;
}

class ConsoleProducer {
    public string Name;
    public List<Console> Consoles;
}

var lines = System.IO.File.ReadAllLines(@"E:\Projects\my_collection\LIST.md");

var gameStart = false;
var gameName = "";
var gameCover = "";

var gameConsole = "";
var gameConsoleMaker = "";

var consoleProducers = new List<ConsoleProducer>();
foreach(var line in lines)
{
    if(line.StartsWith("## "))
        gameConsole = line.Substring(3);
    if(line.StartsWith("# "))
        gameConsoleMaker = line.Substring(2);

    if(!gameStart && line.StartsWith("### "))
    {
        gameName = line.Substring(4);
        gameStart = true;
    }else if(gameStart){
        if(line.StartsWith("![]("))
        {
            var cover = line.Substring(4, line.Length - 5);
            gameCover = cover;
            gameStart = false;

            // Download cover from http
            var gameFileName = Regex.Replace(gameName, @"[^\w\.@-]", "_");
            var gameConsoleFolderName = Regex.Replace(gameConsole, @"[^\w\.@-]", "_");
            var coverFileName = gameFileName + ".jpg";
            if(!System.IO.Directory.Exists(@"E:\Projects\my_collection\covers\"+gameConsoleFolderName))
                System.IO.Directory.CreateDirectory(@"E:\Projects\my_collection\covers\"+gameConsoleFolderName);

            var coverFile = System.IO.Path.Combine(@"E:\Projects\my_collection\covers\"+gameConsoleFolderName, coverFileName);
            if(!System.IO.File.Exists(coverFile))
            {
                try
                {
                    using(var wc = new System.Net.WebClient())
                    {
                        wc.DownloadFile(gameCover, coverFile);
                    }
                }
                catch(System.Exception ex)
                {
                    System.IO.File.WriteAllText(coverFile, "");
                }
            }

            ConsoleProducer consoleProducer = null;
            if(!consoleProducers.Any(c => c.Name == gameConsoleMaker))
            {
                consoleProducer = new ConsoleProducer();
                consoleProducer.Name = gameConsoleMaker;
                consoleProducer.Consoles = new List<Console>();
                consoleProducers.Add(consoleProducer);
            }else{
                consoleProducer = consoleProducers.First(c => c.Name == gameConsoleMaker);
            }

            Console console = null;
            if(consoleProducer.Consoles.Count == 0 || consoleProducer.Consoles.Find(c => c.Name == gameConsole) == null)
            {
                console = new Console();
                console.Name = gameConsole;
                console.Games = new List<Game>();
                consoleProducer.Consoles.Add(console);
            }else{
                console = consoleProducer.Consoles.Find(c => c.Name == gameConsole);
            }

            console.Games.Add(new Game {
                Name = gameName,
                Cover = gameConsoleFolderName + "/" + coverFileName
            });
        }
    }
}

// Convert to JSON file
var json = JsonConvert.SerializeObject(consoleProducers);
json = Regex.Replace(json, @"\'", "\\'");
System.IO.File.Delete(@"E:\Projects\my_collection\games.json");
System.IO.File.WriteAllText(@"E:\Projects\my_collection\games.json", json);